# MA332 Homework 6
#### *Ben Raivel*

In [1]:
import numpy as np
import scipy

Use the Jacobi method and LU-decomposition backsolve implementations from homework 5:

In [2]:
def jacobi(A, b, x=None, tol=1e-6, precision=np.float32, verbose=False):
    """Approximate the solution x to Ax = b using the Jacobi method
    
    Parameters:
    -----------
    - A: (n x n) matrix
    - b: (n x 1) vector
    - x (optional): Initial guess
    - tol: Forward error tolerance
    - precision: Type of float to use. For large matrices, using 16 or
        32 bits can be more stable/faster.
    - verbose: Whether to print the number of iterations taken.
    """
    # using 16 bit precision helps stop python from crashing
    # particularly on Problem 4
    if (A.dtype != precision.__name__):
        A = A.astype(precision)
        b = b.astype(precision)

    if x is None:  # make an initial guess (zeros)
        x = np.zeros(b.shape, dtype=precision)
    x_prev = np.zeros(b.shape, dtype=precision)
    
    # get diagonal of A and its inverse
    d = np.diag(A)
    d_inv = (1/d)

    # get D and D_inv as matrices
    D = d*np.eye(len(A), dtype=precision)
    D_inv = d_inv*np.eye(len(A), dtype=precision)

    # find (L + U)
    LU = A - D

    iter = 0
    error = np.inf

    # iterate until error is less than tol
    while error > tol:
        x_prev = x

        # update x
        x = np.dot(D_inv, b - np.dot(LU, x_prev))
        
        # calculate forward error
        error = np.linalg.norm(x - x_prev, ord=np.inf)

        iter += 1

    if verbose:
        print(f'Found x in {iter} iterations')

    return x

In [3]:
def lu_backsolve(P, L, U, b):
    """Solve Ax = b given the LU-decomposition of A (PA = LU)
    
    Parameters:
    -----------
    - P: Permutation matrix, holds row-swap information
    - L: Lower triangular component
    - U: Upper triangular component
    - b: (n x 1) vector
    """

    # apply inverse permutation matrix to b
    Pb = np.dot(P.T, b)

    # arrays to hold x, y values
    y = np.zeros(len(L))
    x = np.zeros(len(U))

    # forward substitution
    for i in range(len(L)):
        y[i] = (Pb[i] - np.dot(L[i, :i], y[:i]))/L[i, i]

    # backward substitution
    for i in reversed(range(len(U))):
        x[i] = (y[i] - np.dot(U[i, i:], x[i:]))/U[i, i]

    return x

## Problem 1.
#### Gauss-Seidel method implementation

In [28]:
def gauss_seidel(A, b, x=None, tol=1e-6, max_iter=300, verbose=False):
    """Approximate the solution x to Ax = b using the Gauss-Seidel method
    
    Parameters:
    -----------
    - A: (n x n) matrix
    - b: (n x 1) vector
    - x (optional): Initial approximation
    - tol: Forward error tolerance
    - max_iter: Maximum number of iterations
    - verbose: Whether to print the number of iterations taken
    """
    if not np.equal(*A.shape):
        raise Exception('A must be an (n x n) matrix')

    n, n = A.shape

    if x is None:
        x = np.zeros((n, 1))
    
    d = np.diag(A)  # diagonal elements of A
    R = A - d*np.eye(n)  # A with 0s on the diagonal
    
    error = np.inf
    iter = 0

    # iterate until error is less than tol
    while error > tol:

        x_prev = np.copy(x)

        # update x in place
        for i in range(n):
            x[i] = (b[i] - np.dot(R[i], x))/d[i]

        # calculate forward error
        error = np.linalg.norm(x - x_prev, np.inf)

        iter += 1
        if iter >= max_iter: break

    if verbose:
        print(f'Found x in {iter} iterations')

    return x

## Problem 2.

### (a)
The coefficient matrix is:
$$ A =
\begin{pmatrix}
1 & 0 & -1\\
-0.5 & 1 & -0.25\\
1 & -0.5 & 1
\end{pmatrix}
$$
The matrix is strictly diagonally dominant because the elements on the diagonal are greater than the sum of the other elements in the row for every row in the matrix.

In [29]:
A = np.array([[1, 0, -1],
              [-0.5, 1, -0.25],
              [1, -0.5, 1]])

b = np.array([[0.2],
              [-1.425],
              [2]])

### (b)
The matrix form of the Jacobi method is:
$$x^{k + 1} = -D^{-1}(L + U)x^k + D^{-1}b$$
so 
$$T_J = -D^{-1}(L + U)$$

In [30]:
P, L, U = scipy.linalg.lu(A)
D_inv = (1/np.diag(A))*np.eye(len(A))

Tj = -D_inv @ (L + U)

The matrix form of the Gauss-Seidel method is
$$x^{k + 1} = -L^{-1}Ux^k + L^{-1}b$$
so
$$T_G = -L^{-1}U$$

In [31]:
L = np.tril(A)
U = np.triu(A, k=1)

Tg = -np.linalg.inv(L) @ U

The spectral radius is the maximum absolute eigenvalue of $T$

In [32]:
sr_Tj = np.max(np.abs(np.linalg.eigvals(Tj)))
sr_Tg = np.max(np.abs(np.linalg.eigvals(Tg)))

print(f'Spectral radii:\nT_J = {sr_Tj}\nT_G = {sr_Tg}')

Spectral radii:
T_J = 2.588246843315691
T_G = 0.625


### (c) Jacobi method

In [33]:
x_j = jacobi(A, b, tol=1e-2, verbose=True)
print(x_j)

Found x in 187 iterations
[[ 0.9022264 ]
 [-0.79595244]
 [ 0.69281316]]


### (d) Gauss-Seidel method

In [34]:
x_g = gauss_seidel(A, b, tol=1e-2, verbose=True)
print(x_g)

Found x in 13 iterations
[[ 0.8975131 ]
 [-0.80186517]
 [ 0.70155431]]


### (e)
The number of iterations taken by the Jacobi and Gauss-Seidel methods aligns with the spectral radii of $T_J$ and $T_G$. The matrix for the Gauss-Seidel method had a spectral radius of $0.625$ (less than one) and the method converges quickly, finishing in $13$ iterations. The matrix for the Jacobi method had a spectral radius $\sim 2.59$ (greater than one) and takes considerably longer— $187$ iterations— to complete.

## Problem 3.

Define A and b with $n = 100$ and $n = 10,000$:

In [11]:
A_1e2 = (np.diag(3*np.ones(int(1e2)))
         + np.diag(-np.ones(int(1e2 - 1)), -1)
         + np.diag(-np.ones(int(1e2 - 1)), 1))

b_1e2 = np.ones(int(1e2))
b_1e2[0] += 1
b_1e2[-1] += 1

A_1e4 = (np.diag(3*np.ones(int(1e4)))
         + np.diag(-np.ones(int(1e4 - 1)), -1)
         + np.diag(-np.ones(int(1e4 - 1)), 1))

b_1e4 = np.ones(int(1e4))
b_1e4[0] += 1
b_1e4[-1] += 1

### (a) Jacobi method

Jacobi method with $n = 100$

In [12]:
x_1e2 = jacobi(A_1e2, b_1e2, verbose=True)

backward_error = np.linalg.norm(b_1e2 - np.dot(A_1e2, x_1e2), np.inf)
print(f'Backward error: {backward_error:{6}.{2}}')

Found x in 33 iterations
Backward error: 1.5e-06


Jacobi method with $n = 10,000$

In [13]:
x_1e4 = jacobi(A_1e4, b_1e4, verbose=True)

backward_error = np.linalg.norm(b_1e4 - np.dot(A_1e4, x_1e4), np.inf)
print(f'Backward error: {backward_error:{6}.{2}}')

Found x in 33 iterations
Backward error: 1.5e-06


### (b) Gauss-Seidel method

Gauss-Seidel method with $n = 100$

In [14]:
x_1e2 = gauss_seidel(A_1e2, b_1e2, verbose=True)

backward_error = np.linalg.norm(b_1e2 - np.dot(A_1e2, x_1e2), np.inf)
print(f'Backward error: {backward_error:{6}.{2}}')

Found x in 20 iterations
Backward error: 9.6e-07


Gauss-Seidel method with $n = 10,000$

In [15]:
x_1e4 = gauss_seidel(A_1e4, b_1e4, verbose=True)

backward_error = np.linalg.norm(b_1e4 - np.dot(A_1e4, x_1e4), np.inf)
print(f'Backward error: {backward_error:{6}.{2}}')

Found x in 20 iterations
Backward error: 9.6e-07


### (c) L-U decomposition and backsolve

LU-decomposition with $n = 100$

In [16]:
P_1e2, L_1e2, U_1e2 = scipy.linalg.lu(A_1e2)
x_1e2 = lu_backsolve(P_1e2, L_1e2, U_1e2, b_1e2)

backward_error = np.linalg.norm(b_1e2 - np.dot(A_1e2, x_1e2), np.inf)
print(f'Backward error: {backward_error:{6}.{2}}')

Backward error: 8.9e-16


LU-decomposition with $n = 10,000$

In [17]:
P_1e4, L_1e4, U_1e4 = scipy.linalg.lu(A_1e4)
x_1e4 = lu_backsolve(P_1e4, L_1e4, U_1e4, b_1e4)

backward_error = np.linalg.norm(b_1e4 - np.dot(A_1e4, x_1e4), np.inf)
print(f'Backward error: {backward_error:{6}.{2}}')

Backward error: 8.9e-16


## Problem 4.

Linear system in matrix form:
$$
\begin{pmatrix}
1 & 1\\
1.0001 & 1
\end{pmatrix}
\begin{pmatrix}
x_1\\
x_2
\end{pmatrix} =
\begin{pmatrix}
2\\
2.0001
\end{pmatrix}
$$

In Python:

In [18]:
A = np.array([[1, 1],
              [1.0001, 1]])

b = np.array([[2],
              [2.0001]])

x_actual = np.array([[1],
                     [1]])

### (a)

Compute forward error:

In [19]:
x_approx = np.array([[2],
                     [2]])

forward_error  = np.linalg.norm(x_actual - x_approx, np.inf)
print(f'Forward error: {forward_error}')

Forward error: 1.0


Compute backward error:

In [20]:
backward_error = np.linalg.norm(b - A @ x_approx, np.inf)
print(f'Backward error: {backward_error}')

Backward error: 2.0000999999999993


Compute error magnification factor:

In [21]:
rel_forward_error = forward_error/np.linalg.norm(x_actual, np.inf)
rel_backward_error = backward_error/np.linalg.norm(b, np.inf)
error_mag = rel_forward_error/rel_backward_error

print(f'Error magnification factor: {error_mag}')

Error magnification factor: 1.0000000000000004


### (b)

The condition number $\kappa(A)$ is given by:

$$
\kappa(A) = \frac{\text{max}(\text{abs}(\text{eig}(A)))}{\text{min}(\text{abs}(\text{eig}(A)))}
$$

In [27]:
condition_A = np.max(np.abs(np.linalg.eigvals(A)))/np.min(np.abs(np.linalg.eigvals(A)))

print(f'Condition number of A: {condition_A:{7}.{7}}')

Condition number of A: 40002.0
